In [1]:
import pandas as pd
import numpy as np
from torch.utils.data import Dataset, DataLoader
import os


from src.data_loader import *

In [2]:
class Transform:

    def __init__(self, df, method = 'simple', window_size= 1):
        self.method = method
        self.df = df
        self.signal = []
        self.window_size = window_size
        self.label_dict = {0:'매수', 1:'매도', 2:'유지'}

    def simple_generator(self):

        for idx in range(len(self.df['Adj Close'])):
            if idx == 0 or idx+1 == len(self.df['Adj Close']):
                self.signal.append(0)   
            else:
                if self.df['Adj Close'].iloc[idx-1]> self.df['Adj Close'].iloc[idx] and \
                        self.df['Adj Close'].iloc[idx+1]> self.df['Adj Close'].iloc[idx]:
                    self.signal.append(0)
                elif self.df['Adj Close'].iloc[idx-1]< self.df['Adj Close'].iloc[idx] and \
                        self.df['Adj Close'].iloc[idx+1]< self.df['Adj Close'].iloc[idx]:
                    self.signal.append(1)
                else:
                    self.signal.append(2)
        self.df['signal'] = self.signal
        return self.df.drop('signal', axis = 1), self.df['signal']
    

    def time_window(self, data): 
        '''
        df          : 변환할 데이터
        window_size : 분할 크기  
        '''

        result = []
        for index in range(len(self.df) - self.window_size):
            result.append(self.df[index: index + self.window_size].values)

        return np.array(result)

    def execute(self):
        if self.method == 'simple':
            X, y = self.simple_generator()
            return self.time_window(X), self.time_window(y)

In [3]:
class StockDataset(Dataset):
    def __init__(self, path, date_type, interval = 'day', ticker = None, window_size = 20,transform=None, method = 'simple'):
        self.ticker = ticker
        self.method = method
        self.df = DataGenerator(date_type, interval).data_search(ticker = self.ticker)
        self.X, self.y = Transform(df = self.df, method = self.method).execute()
        self.X = self.X.astype(np.float32)
        self.y = self.y.astype(np.int64)

    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, idx):
        # if self.transform is not None:
        #     data = YGenerator(df = self.df, method = self.transform)
        return self.X[idx], self.y[idx]

In [51]:
dataset = StockDataset(path = './Dataset',date_type = 'db', interval='day', ticker = 'GOOG' )
dataloader = DataLoader(dataset=dataset,
                        batch_size=100,
                        shuffle=False,
                        drop_last=False)

DB에서 데이터 불러오기 성공
